# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [18]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import datetime
import os
#import matplotlib.pyplot as plt
from skimage import io
from skimage.transform import rescale, resize, downscale_local_mean

In [19]:
import matplotlib.pyplot as plt

We set the random seed so that the results don't vary drastically.

In [20]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [21]:
data_path = './Project_data'
train_path = f'{data_path}/train'
val_path = f'{data_path}/val'


In [22]:
train_doc = np.random.permutation(open(f'{data_path}/train.csv').readlines())
val_doc = np.random.permutation(open(f'{data_path}/val.csv').readlines())
batch_size = 5 #experiment with the batch size

In [23]:
train_doc[0:10]

array(['WIN_20180926_16_54_08_Pro_Right_Swipe_new;Right_Swipe_new;1\n',
       'WIN_20180925_18_02_58_Pro_Thumbs_Down_new;Thumbs_Down_new;3\n',
       'WIN_20180925_17_33_08_Pro_Left_Swipe_new;Left_Swipe_new;0\n',
       'WIN_20180925_17_51_17_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n',
       'WIN_20180926_17_17_35_Pro_Left_Swipe_new;Left_Swipe_new;0\n',
       'WIN_20180926_17_30_47_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n',
       'WIN_20180926_17_29_23_Pro_Stop_new;Stop_new;2\n',
       'WIN_20180926_17_12_26_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n',
       'WIN_20180926_17_29_01_Pro_Stop_new;Stop_new;2\n',
       'WIN_20180907_16_32_11_Pro_Stop Gesture_new;Stop Gesture_new;2\n'],
      dtype='<U88')

In [24]:
len(train_doc) // batch_size

132

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [25]:
from skimage import transform as tf
from skimage.util import random_noise
from skimage import exposure
from skimage.transform import rotate as rot

def normalize(image):
#     return image/255
#    return image - np.percentile(image,5)/ np.percentile(image,95) - np.percentile(image,5)
     return image - np.min(image)/np.max(image) - np.min(image)

def identity(image):
    return image

def flip_lr(image):
    return np.fliplr(image)

def inc_contrast(image):
    v_min, v_max = np.percentile(image, (0.2, 99.8))
    return exposure.rescale_intensity(image, in_range=(v_min, v_max))

def inc_brightness(image):
    return exposure.adjust_gamma(image, gamma=0.4, gain=0.9)

def bright_jitter(image):
    return image*0.999 + np.zeros_like(image)*0.001

def noise(image):
    return random_noise(image)

def rotate(image):
    return rot(image, rn.choice(range(5,20)))

def rotate_anti(image):
    return rot(image, rn.choice(range(-20,-5)))

def shift_left(image):
    #M = np.float32([[1, 0, -25], [0, 1, 0]])
    # translation
    #(rows, cols) = image.shape[:2] 
    #return cv2.warpAffine(image, M, (cols, rows))
    transaltion_transforms = tf.AffineTransform(translation=(rn.choice(range(15,40)), 0))
    return tf.warp(image, transaltion_transforms)

def shift_right(image):
    transaltion_transforms = tf.AffineTransform(translation=(rn.choice(range(-40,-15)), 0))
    return tf.warp(image, transaltion_transforms)

def shift_up(image):
    transaltion_transforms = tf.AffineTransform(translation=(0, rn.choice(range(-40,-15))))
    return tf.warp(image, transaltion_transforms)

def shift_down(image):
    transaltion_transforms = tf.AffineTransform(translation=(0, rn.choice(range(15, 40))))
    return tf.warp(image, transaltion_transforms)

def zoom_in(image):
    # specify x and y coordinates to be used for shifting (mid points)
    shift_x, shift_y = image.shape[0]/2, image.shape[1]/2

    # translation by certain units
    matrix_to_topleft = tf.SimilarityTransform(translation=[-shift_x, -shift_y])
    matrix_to_center = tf.SimilarityTransform(translation=[shift_x, shift_y])

    zoom = rn.choice(range(60,90,5))/100
    scale_transforms = tf.AffineTransform(scale=(zoom, zoom))
    scale_matrix = matrix_to_topleft + scale_transforms + matrix_to_center
    scale_image_zoom_in = tf.warp(image, scale_matrix)
    return scale_image_zoom_in

def zoom_out(image):
    # specify x and y coordinates to be used for shifting (mid points)
    shift_x, shift_y = image.shape[0]/2, image.shape[1]/2

    # translation by certain units
    matrix_to_topleft = tf.SimilarityTransform(translation=[-shift_x, -shift_y])
    matrix_to_center = tf.SimilarityTransform(translation=[shift_x, shift_y])

    # scaling 
    zoom = rn.choice(range(130,160,5))/100
    scale_transforms = tf.AffineTransform(scale=(zoom, zoom))
    scale_matrix = matrix_to_topleft + scale_transforms + matrix_to_center
    scale_image_zoom_out = tf.warp(image, scale_matrix)

    return scale_image_zoom_out


def feed_image(batch_data, folder, idx, image):
    batch_data[folder,idx,:,:,0] = image[:, :, 0] #normalise and feed in the image
    batch_data[folder,idx,:,:,1] = image[:, :, 1] #normalise and feed in the image
    batch_data[folder,idx,:,:,2] = image[:, :, 2] #normalise and feed in the image

def augment_batch(batch_data, fn):
    new_batch = np.zeros(batch_data.shape)
    (folders, indices,_,_,_) = batch_data.shape
    for folder in range(folders):
        for idx in range(indices):
            for channel in range(3):
                new_batch[folder,idx,:,:,channel] = normalize(fn(batch_data[folder,idx,:,:,channel]))
    return new_batch

def augment_batch_2(batch_data, fns):
    new_batch = np.zeros(batch_data.shape)
    funcs = rn.sample(fns, rn.choice([1, 2, 3]))
    (folders, indices,_,_,_) = batch_data.shape
    for folder in range(folders):
        for idx in range(indices):
            for channel in range(3):
                image = batch_data[folder,idx,:,:,channel] 
                for fn in funcs:
                    image = fn(image)
                new_batch[folder,idx,:,:,channel] = normalize(image)
    return new_batch


def generator(source_path, folder_list, batch_size, augment=False):
    #print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = list(range(0, 30, 3)) #create a list of image numbers you want to use for a particular video
    x = len(img_idx) # number of images for each video
    y = 120          # height of each input image
    z = 120          # width of each input image
    large=(360, 360, 3)
    small=(120, 160, 3)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    #print("Reading ", source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    image = io.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])

                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if (image.shape == large):
                        image = resize(image, (y, z), anti_aliasing=False)
                    else:
                        image = image[:, 20:140, :]
                    
                    feed_image(batch_data, folder, idx, image)
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

            yield augment_batch(batch_data, identity), batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
            if augment:
                #yield augment_batch(batch_data, inc_contrast), batch_labels
                #yield augment_batch(batch_data, inc_brightness), batch_labels
                yield augment_batch(batch_data, bright_jitter), batch_labels
                #yield augment_batch(batch_data, zoom_in), batch_labels
                #yield augment_batch(batch_data, zoom_out), batch_labels
                yield augment_batch_2(batch_data, [inc_contrast, zoom_in, zoom_out, rotate, rotate_anti, shift_left, shift_right, shift_up, shift_down]), batch_labels
                yield augment_batch_2(batch_data, [inc_contrast, zoom_in, zoom_out, rotate, rotate_anti, shift_left, shift_right, shift_up, shift_down]), batch_labels
                #yield augment_batch(batch_data, shift_left), batch_labels
                #yield augment_batch(batch_data, shift_right), batch_labels
                yield augment_batch_2(batch_data, [inc_contrast, zoom_in, zoom_out, rotate, rotate_anti, shift_left, shift_right, shift_up, shift_down]), batch_labels
                yield augment_batch_2(batch_data, [inc_contrast, zoom_in, zoom_out, rotate, rotate_anti, shift_left, shift_right, shift_up, shift_down]), batch_labels
                #yield augment_batch_2(batch_data, [inc_contrast, zoom_in, zoom_out, rotate, rotate_anti, shift_left, shift_right, shift_up, shift_down]), batch_labels
        
        # write the code for the remaining data points which are left after full batches
        start_idx = num_batches * batch_size
        remaining_size = len(folder_list) - start_idx
        if remaining_size > 0:
            batch_data = np.zeros((remaining_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((remaining_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(start_idx, len(folder_list)): #iterate over the remaining folders
                imgs = os.listdir(source_path+'/'+ t[folder].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = io.imread(source_path+'/'+ t[folder].strip().split(';')[0]+'/'+imgs[item])

                    if (image.shape == large):
                        image = resize(image, (y, z), anti_aliasing=False)
                    else:
                        image = image[:, 20:140, :]
                    
                    feed_image(batch_data, folder - start_idx, idx, image)

            batch_labels[folder - start_idx, int(t[folder].strip().split(';')[2])] = 1
            yield augment_batch(batch_data, identity), batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
            if augment: 
                #yield augment_batch(batch_data, inc_contrast), batch_labels
                #yield augment_batch(batch_data, inc_brightness), batch_labels
                yield augment_batch(batch_data, bright_jitter), batch_labels
                yield augment_batch_2(batch_data, [inc_contrast, zoom_in, zoom_out, rotate, rotate_anti, shift_left, shift_right, shift_up, shift_down]), batch_labels
                yield augment_batch_2(batch_data, [inc_contrast, zoom_in, zoom_out, rotate, rotate_anti, shift_left, shift_right, shift_up, shift_down]), batch_labels
                yield augment_batch_2(batch_data, [inc_contrast, zoom_in, zoom_out, rotate, rotate_anti, shift_left, shift_right, shift_up, shift_down]), batch_labels
                yield augment_batch_2(batch_data, [inc_contrast, zoom_in, zoom_out, rotate, rotate_anti, shift_left, shift_right, shift_up, shift_down]), batch_labels
                #yield augment_batch_2(batch_data, [inc_contrast, zoom_in, zoom_out, rotate, rotate_anti, shift_left, shift_right, shift_up, shift_down]), batch_labels
                #yield augment_batch(batch_data, shift_left), batch_labels
                #yield augment_batch(batch_data, shift_right), batch_labels
        
    return None, None


In [26]:
# test generator 
#i = generator(data_path + "/train", train_doc, 32)

In [27]:
# data, labels = next(i)
# print(len(train_doc))
# print(len(data))
# print(len(labels))

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [28]:
curr_dt_time = datetime.datetime.now()
input_image_shape = (10, 120, 120, 3)
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 2 # choose the number of epochs
print ('# epochs =', num_epochs)
num_classes = 5

# training sequences = 663
# validation sequences = 100
# epochs = 2


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [29]:
def make_Conv3D_model():
    #write your model here
    model = Sequential()
    model.add(GaussianNoise(0.01, input_shape=input_image_shape))
    
    model.add(Conv3D(32, (3, 3, 3), padding="same", input_shape=input_image_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv3D(32, (3, 3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv3D(64, (3, 3, 3), padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv3D(64, (3, 3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(16, kernel_regularizer=l2(0.01)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model

def make_ConvLSTM_model():
    lstm_input_shape = (10, 120, 120, 3)
    
    #write your model here
    model = Sequential()
    model.add(GaussianNoise(0.01, input_shape=lstm_input_shape))
    model.add(ConvLSTM2D(16, (3, 3), padding="valid",
                     input_shape=lstm_input_shape, return_sequences=True))
    model.add(Activation('relu'))
#    model.add(BatchNormalization())
#     model.add(ConvLSTM2D(32, (3, 3), return_sequences=True))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#    model.add(Dropout(0.25))

#    model.add(ConvLSTM2D(64, (3, 3), padding="valid", return_sequences=True))
#    model.add(Activation('relu'))
#    model.add(BatchNormalization())
#    model.add(ConvLSTM2D(64, (3, 3), return_sequences=True))
#    model.add(Activation('relu'))
#    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(32, kernel_regularizer=l2(0.01)))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model

def make_CNN_LSTM_model():
    model = Sequential()
    model.add(GaussianNoise(0.01, input_shape=input_image_shape))
    # define CNN model
    model.add(TimeDistributed(Conv2D(24, (3, 3), padding="same", activation='relu'), input_shape=input_image_shape))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    #model.add(TimeDistributed(Conv2D(24, (3, 3), padding="same", activation='relu')))
    #model.add(TimeDistributed(BatchNormalization()))
    #model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
#     model.add(TimeDistributed(Dropout(0.5)))
#     model.add(TimeDistributed(Conv2D(64, (3, 3), padding="same", activation='relu')))
#     model.add(TimeDistributed(BatchNormalization()))
#     model.add(TimeDistributed(Conv2D(64, (3, 3), padding="same", activation='relu')))
#     model.add(TimeDistributed(BatchNormalization()))
#     model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Flatten()))
    model.add(Dropout(0.2))
    # define LSTM model
    model.add(LSTM(32, return_sequences=True))
#     model.add(TimeDistributed(Dropout(0.5)))
#     model.add(LSTM(25, return_sequences=True))
#     model.add(TimeDistributed(Dense(32, activation='relu', kernel_regularizer=l2(0.01))))
    model.add(TimeDistributed(Dropout(0.2)))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    return model


In [30]:
def run_Conv3D_model():
    conv3DModel = make_Conv3D_model()
    process_model(conv3DModel, train_path, train_doc, batch_size, val_path, val_doc, num_train_sequences, num_val_sequences)

In [31]:
def run_ConvLSTM_model():
    convLSTMModel = make_ConvLSTM_model()
    process_model(convLSTMModel, train_path, train_doc, batch_size, val_path, val_doc, num_train_sequences, num_val_sequences)

In [32]:
def run_CNNLSTM_model():
    cnnLSTMModel = make_CNN_LSTM_model()
    process_model(cnnLSTMModel, train_path, train_doc, batch_size, val_path, val_doc, num_train_sequences, num_val_sequences)

In [33]:
def process_model(model, train_path, train_doc, batch_size, val_path, val_doc, num_train_sequences, num_val_sequences):
    optimiser = optimizers.SGD(lr=0.01, clipnorm=1.) #write your optimizer
    #optimiser = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    print (model.summary())
    
    #Let us create the train_generator and the val_generator which will be used in .fit_generator.
    train_generator = generator(train_path, train_doc, batch_size, True)
    val_generator = generator(val_path, val_doc, batch_size)
    
    model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

    if not os.path.exists(model_name):
        os.mkdir(model_name)

    filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

    LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                  patience=3, min_lr=0.001) # write the REducelronplateau code here
    callbacks_list = [checkpoint, LR]    
    
    #The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() 
    #calls it need to make.
    if (num_train_sequences%batch_size) == 0:
        steps_per_epoch = int(num_train_sequences/batch_size)
    else:
        steps_per_epoch = (num_train_sequences//batch_size) + 1

    steps_per_epoch = steps_per_epoch * 6 #Because we are augmenting each image 

    if (num_val_sequences%batch_size) == 0:
        validation_steps = int(num_val_sequences/batch_size)
    else:
        validation_steps = (num_val_sequences//batch_size) + 1    
        
    #Let us now fit the model. This will start training the model and with the help of the checkpoints, 
    #you'll be able to save the model at the end of each epoch. 
    model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                        callbacks=callbacks_list, validation_data=val_generator, 
                        validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)    

In [34]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, ConvLSTM2D, GaussianNoise, LSTM
from keras.layers.convolutional import Conv2D, Conv3D, MaxPooling3D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.regularizers import l2
from keras import optimizers

#write your model here
#model = make_Conv3D_model()
#model = make_ConvLSTM_model()
#model = make_CNN_LSTM_model()

run_Conv3D_model()
run_ConvLSTM_model()
run_CNNLSTM_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_noise_2 (GaussianNo (None, 10, 120, 120, 3)   0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 10, 120, 120, 32)  2624      
_________________________________________________________________
activation_7 (Activation)    (None, 10, 120, 120, 32)  0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 10, 120, 120, 32)  128       
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 8, 118, 118, 32)   27680     
_________________________________________________________________
activation_8 (Activation)    (None, 8, 118, 118, 32)   0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 8, 118, 118, 32)   128       
__________

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


798/798 [==============================] - 272s 341ms/step - loss: 2.0226 - categorical_accuracy: 0.2321 - val_loss: 2.0705 - val_categorical_accuracy: 0.2200

Epoch 00001: saving model to model_init_2019-06-2407_30_03.745145/model-00001-2.02622-0.23278-2.07048-0.22000.h5
Epoch 2/2
798/798 [==============================] - 270s 339ms/step - loss: 1.8998 - categorical_accuracy: 0.2406 - val_loss: 1.9182 - val_categorical_accuracy: 0.2000

Epoch 00002: saving model to model_init_2019-06-2407_30_03.745145/model-00002-1.90304-0.24133-1.91820-0.20000.h5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_noise_3 (GaussianNo (None, 10, 120, 120, 3)   0         
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 10, 118, 118, 16)  11008     
_________________________________________________________________
activation_13 (Activation)   (None, 10, 118, 118